In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import io
import requests
import datetime
import ondemand
from scipy.stats import pearsonr
from pandas.io.common import EmptyDataError
from requests.exceptions import HTTPError
import time
import os
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [98]:
#Pick the top 10 symbols with highest PageView_Change_in_std and PageView > 10
"""
Filters:
<1> Remove stocks with a gap up or down day within the window period
<2> Remove stocks that have a significantly greater high low range within the window period compared to normal.
"""
def my_filters(rawdata, date, win):
    symbols = list(rawdata['Symbol'])
    url_price = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=OnDemand&symbol={}&type=daily&startDate={}&endDate={}&maxRecords=1000&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration'
    temp_win_date = datetime.datetime.strptime(date, "%Y-%m-%d")
    count = win
    window_days = []
    while count > 0:
        try:
            pd.read_csv(url_price.format('AAPL', temp_win_date, temp_win_date)) # test trading day
            window_days.append(temp_win_date)
            temp_win_date -= datetime.timedelta(days = 1)
            count -= 1
        except EmptyDataError:
            temp_win_date -= datetime.timedelta(days = 1)
        except ConnectionResetError:
            print('API problem')
            continue
    window_days = window_days[::-1]
    win_start = window_days[0]
    win_end = window_days[-1]
    shock_start = win_start
    count2 = 20
    while count2 > 0:
        shock_start -= datetime.timedelta(days = 1)
        try:
            pd.read_csv(url_price.format('AAPL', shock_start, shock_start)) # test trading day
            count2 -= 1
        except EmptyDataError:
            continue
        except ConnectionResetError:
            print('API problem')
            continue
    win_start = str(win_start)[:10]
    #shock_start = str(shock_start)[:10]
    to_drop = []
    for item in symbols:
        price = pd.read_csv(url_price.format(item, shock_start, date))
        price['high_loc_diff'] = price['high'] - price['low']
        price['H_L_20day_moving_avg'] = price['high_loc_diff'].rolling(20).mean()
        try:
            high_low_benchmark = price.iloc[19, 10]
        except IndexError:
            print(price)
        win_low = 0
        win_high = 0
        for each in window_days:
            temp_price = price.loc[price['tradingDay'] == str(each)[:10]]
            try:
                last_day_index = temp_price.index.item() - 1
            except:
                print(temp_price)
            last_day_price = price.iloc[last_day_index]
            if float(temp_price['open']) > float(last_day_price['high']) or\
            float(temp_price['open']) < float(last_day_price['low']):
                to_drop.append(item)
            if win_low == 0:
                win_low = float(temp_price['low'])
            else:
                win_low = min(float(temp_price['low']), win_low)
            if win_high == 0:
                win_high = float(temp_price['high'])
            else:
                win_high = max(float(temp_price['high']), win_high)
        win_high_low = float(win_high - win_low)
        if win_high_low > 2.5*high_low_benchmark:
            to_drop.append(item)
    to_drop = list(set(to_drop))
    print(to_drop)
    for item in to_drop:
        rawdata = rawdata.loc[rawdata['Symbol'] != item]
    return rawdata
for win in [1, 3]:
    for date in daterange1:
        for cap in ['mid']:
            try:
                to_filter = pd.read_csv\
                ('leaderlist {}/{}_cap/window_{}_day/{}_cap_highest_change_time_window_{}.csv'.format(date, cap, win, cap, win))
            except FileNotFoundError:
                continue
            to_filter = to_filter.head(30)
            to_filter = my_filters(to_filter, date, win)
            each = date
            each = each[5:]
            each = each.replace('-', '')
            to_filter = to_filter.loc[to_filter['PageView'] > 10]
            to_filter = to_filter.head(10)
            to_filter.to_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win), index = False)
            

['HAE', 'CADE', 'WFT', 'TSRO', 'CMC', 'CRSP', 'WRE', 'NFG', 'IDCC', 'UNF', 'SMG', 'KL']
['DGRW', 'TMHC', 'DORM', 'RPM', 'TPL', 'GTLS', 'BB', 'TRQ', 'HHC', 'CMC', 'ROG', 'ARNA', 'ITRI', 'CNX', 'PAH', 'EEFT', 'DCP', 'ARES', 'HRC']
['BB', 'LAZ', 'OMF', 'APPN', 'GTLS', 'G', 'QGEN', 'FNB', 'RS', 'TEN', 'SRCL', 'VVV', 'SMG', 'RPM', 'LB']
['TMHC', 'GNTX', 'TELL', 'LAZ', 'CALM', 'AMGP', 'PSMT', 'AMRN', 'CRSP', 'TLND', 'MNTA', 'BBU']
['ARW', 'FND', 'CNX', 'SWN', 'PSMT', 'HCM', 'R', 'AIT', 'CMC', 'GSH', 'ERF', 'CRSP', 'NFG', 'VLY', 'UNF', 'VVV', 'SMG', 'BTU', 'ARES']
['DORM', 'TPL', 'HHC', 'PSMT', 'ITRI', 'VVV', 'COMM', 'BB', 'GTLS', 'SBGI', 'R', 'ERF', 'CRSP', 'HCM', 'CMC', 'ROG', 'ARNA', 'SMG', 'ARES', 'TMHC', 'WFT', 'TSRO', 'NFG', 'CNX', 'UNF', 'DCP']
['DORM', 'HHC', 'ITRI', 'VVV', 'COMM', 'BB', 'GTLS', 'LAZ', 'G', 'FNB', 'R', 'CRSP', 'OMF', 'RRC', 'CMC', 'ARNA', 'SMG', 'RPM', 'ARES', 'TMHC', 'WFT', 'TSRO', 'WRE', 'NFG', 'UNF', 'DCP']
['DORM', 'TPL', 'HHC', 'PSMT', 'ITRI', 'BB', 'GTLS', 'LAZ'

In [100]:
#Generate trading detail for 1-business day
for win in [3]:
    for date in daterange1:
        print(date)
        for cap in ['mid']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = datetime.datetime.strptime(date, "%Y-%m-%d")
            trading_day += datetime.timedelta(days = 1)
            myflag = True
            while myflag:
                try:
                    temp = pd.read_csv(url.format('AAPL', trading_day, trading_day))
                    myflag = False
                except EmptyDataError:
                    trading_day += datetime.timedelta(days = 1)
                except ConnectionResetError:
                    print('API problem')
                    continue
            trading_day = str(trading_day)[:10]
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            result = pd.DataFrame([], columns = ['symbol', 'win_start', 'win_end', 'actual_pageview', 'average_pageview', \
                                                 'std_100days_rolling', 'actual_pageview_in_std', 'buy_date', 'sell_date', \
                                                 'buy_price', 'sell_price', 'p&l', 'pct_change(%)', 'window'])
            temp_win_date = datetime.datetime.strptime(date, "%Y-%m-%d")
            count = win
            window_days = []
            while count > 0:
                try:
                    pd.read_csv(url_price.format('AAPL', temp_win_date, temp_win_date)) # test trading day
                    window_days.append(temp_win_date)
                    temp_win_date -= datetime.timedelta(days = 1)
                    count -= 1
                except EmptyDataError:
                    temp_win_date -= datetime.timedelta(days = 1)
                except ConnectionResetError:
                    print('API problem')
                    continue
            window_days = window_days[::-1]
            win_start = window_days[0]
            win_end = window_days[-1]
            result['symbol'] = portfolio
            result['win_start'] = str(win_start)[:10]
            result['win_end'] = str(win_end)[:10]
            result['buy_date'] = trading_day
            result['sell_date'] = trading_day
            for i, item in enumerate(portfolio):
                flag = True
                while flag:
                    try:
                        price = pd.read_csv(url.format(item, trading_day, trading_day))
                        flag = False
                    except ConnectionResetError:
                        print('API problem')
                        continue
                pageview_price = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(item))
                temp = pageview_price['PageView']
                pageview_price['std_100days_rolling'] = (temp.rolling(100).std()).shift(win)
                total_pageview = 0
                for each in window_days:
                    total_pageview += int(pageview_price.loc[pageview_price['Date'] == str(each)[:10]]['PageView'])
                result.loc[i, 'actual_pageview'] = total_pageview
                result.loc[i, 'average_pageview'] = total_pageview/win
                result.loc[i, 'std_100days_rolling'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['std_100days_rolling'])
                result.loc[i, 'actual_pageview_in_std'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['PageView_Change_in_Std_{}'.format(win)])
                result.loc[i, 'buy_price'] = price.loc[0, 'open']
                result.loc[i, 'sell_price'] = price.loc[0, 'close']
                result.loc[i, 'p&l'] = price.loc[0, 'close'] - price.loc[0, 'open']
                result.loc[i, 'pct_change(%)'] = (price.loc[0, 'close']/price.loc[0, 'open'] - 1)*100
                result.loc[i, 'window'] = win
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_1_day/{}_{}_cap_window_{}_hold_1_detail.csv'.format(date, cap, win, trading_each, cap, win), index = False)
            
            
            

2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
